# Example 07: Upsampling calculations

In this example, upsampling calculations which can be used to switch a system between two different interatomic potentials is illustrated. 

The major change in the input file is that both `pair_style` and `pair_coeff` keywords have two arguments. These are the two potentials between which the transformation will be carried out.

The first potential is a Finnis-Sinclair (FS) potential for copper:  

[M.I. Mendelev, M.J. Kramer, C.A. Becker, and M. Asta (2008), "Analysis of semi-empirical interatomic potentials appropriate for simulation of crystalline and liquid Al and Cu", Philosophical Magazine, 88(12), 1723-1750.](https://doi.org/10.1080/14786430802206482)

The second potential is an EAM:  

[Mishin, Y., M. J. Mehl, D. A. Papaconstantopoulos, A. F. Voter, and J. D. Kress. “Structural Stability and Lattice Defects in Copper: Ab Initio , Tight-Binding, and Embedded-Atom Calculations.” Physical Review B 63, no. 22 (May 21, 2001): 224106.](https://doi.org/10.1103/PhysRevB.63.224106)  

If we know the free energy of the FS potential at a given temperature, we can calculate the free energy of the EAM through upsampling calculations. Upsampling calculations generally need only much less switching time, thus is quite useful in the case of expensive interatomic potentials, such as machine learning potentials.

We start by calculating the free energy of the FS potential. The input file is available at `pot1/input.1.yaml`. As usual, the calculation can be run using `calphy -i input.1.yaml`. We can now load the `report.yaml` file and check the free energy.

In [1]:
import numpy as np
from calphy.postprocessing import read_report

In [2]:
pot1 = read_report('pot1/fe-fcc-solid-600-0')

In [3]:
pot1['results']['free_energy']

-3.4389380251304913

Now we can transform FS to EAM potential (see input file above). After running the calculation, we can check the free energy of this alchemical transformation.

In [4]:
alchemy = read_report('alchemy-fcc-solid-600-0')

In [5]:
alchemy['results']['free_energy']

-0.25451608590151586

The free energy of the EAM potential, $F_\mathrm{EAM} = F_\mathrm{FS} + F_\mathrm{upsampling}$

In [6]:
pot1['results']['free_energy']+alchemy['results']['free_energy']

-3.693454111032007

We can verify this calculation by directly computing the free energy of the EAM potential. The input file is available at `pot2/input-fe.yaml`. 

In [7]:
pot2 = read_report('pot2/fe-fcc-solid-600-0')

In [8]:
pot2['results']['free_energy']

-3.6923377327832667

We can see that the directly calculation is in meV agreement with the upsampling calculations.